In [22]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)

public prompt

In [23]:
import pandas as pd

df = pd.read_csv("./data/0408/data/data_final.csv")

candidates_df = df[(df['rewritten_text'].str.split(" ").apply(len) <= 60 ) & (df['original_text'].str.split(" ").apply(len) <= 30)]
candidates_df

general_df = df[df['type'] == 'general']
literary_df = df[df['type'] == 'literary']
specific_df = df[df['type'] == 'specific']
modern_df = df[df['type'] == 'modern']

print(
    general_df.shape,
    literary_df.shape,
    specific_df.shape,
    modern_df.shape,
)

(249, 4) (274, 4) (263, 4) (232, 4)


dung's prompt

In [24]:
from sklearn.cluster import KMeans
import numpy as np

def diverse_subset(vectors, k):
    # Convert vectors to numpy array
    X = np.array(vectors)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    # Get the cluster labels
    cluster_labels = kmeans.labels_
    
    # Find the centroids of each cluster
    centroids = kmeans.cluster_centers_
    
    # Initialize a list to store the diverse subset
    diverse_subset = []
    
    # Select one vector from each cluster to include in the diverse subset
    for i in range(k):
        cluster_indices = np.where(cluster_labels == i)[0]
        if len(cluster_indices) > 0:
            # Choose a representative vector from the cluster (e.g., centroid)
            representative_index = np.argmin(np.linalg.norm(X[cluster_indices] - centroids[i], axis=1))
            diverse_subset.append(cluster_indices[representative_index])
    
    return diverse_subset

# subset = diverse_subset(embeddings, 4)
# print("Diverse subset:", len(subset))

In [31]:
# encode
from sentence_transformers import SentenceTransformer

t5_base = SentenceTransformer('sentence-t5-base')

ls = []
for df in [general_df, literary_df, specific_df, modern_df]:
    embeddings = t5_base.encode(df['rewrite_prompt'].tolist()).tolist()

    subset = diverse_subset(embeddings, 4)
    data = df.iloc[subset].reset_index(drop=True)
    ls.append(data)

total_df = pd.concat(ls)

total_df.to_csv("./data/0408/data/categorized_seed.csv", index=False)

for i, row in total_df.iterrows():
    print(f"\n======{i}\n")
    print(f'''Given this passage.
"""{row['original_text']}"""

After applying this rewrite prompt "{row['rewrite_prompt']}", the passage is rewritten to this:
"""{row['rewritten_text']}"""

Explain how the above rewritten passage aligns with rewrite prompt and original given passage. Let's think step by step. Keep it short and concise, less than 60 words.
Your result must be in angle of rewrite prompt, such as: "Comparing the rewritten text to original text, the rewrite prompt must''')


/opt/miniconda3/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/miniconda3/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/miniconda3/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



======0

Given this passage.
"""David strolled down the bustling boulevard, mesmerized by the vibrant lights and the lively atmosphere. It was a place where dreams came alive, and he couldn't help but feel alive too."""

After applying this rewrite prompt "Revamp this for a historical context.", the passage is rewritten to this:
""""A steady twilight bathed the bustling boulevard, casting an ethereal glow upon the opulent architecture and bustling crowd. It was a scene of vibrant life, where aspirations soared and hope danced. David marveled at the spectacle, his heart soaring with the electricity of the moment. The boulevard represented a confluence of dreams, a melting pot of aspirations and aspirations.""""

Explain how the above rewritten passage aligns with rewrite prompt and original given passage. Let's think step by step. Keep it short and concise, less than 60 words.
Your result must be in angle of rewrite prompt, such as: "Comparing the rewritten text to original text, the r

/opt/miniconda3/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [34]:
reasoning_ls = [
    "Comparing the rewritten text to original text, the rewrite prompt must alter the text by intensifying sensory details and emotions, creating a vivid scene. It retains Irene's interaction with her cat during a thunderstorm, enhancing it with evocative imagery and deeper emotional resonance.",
    "Comparing the rewritten text to original text, the rewrite prompt must alter the text by intensifying the tone and adding urgency, conveying concern in a more assertive manner. It maintains the core elements of expressing care and offering assistance, albeit with a stronger emphasis on urging action and seeking support.",
    "Comparing the rewritten text to the original text, the rewrite prompt enhances engagement by transforming the description into a vivid narrative with descriptive language, sensory imagery, and metaphorical comparisons, making Martin's room feel more alive and inviting to the reader.",
    "Comparing the rewritten text to the original text, the rewrite prompt transforms the passage into a journalistic-style narrative by providing detailed descriptions, emphasizing key events, and incorporating quotes or observations from relevant parties. It adds depth and context to Marilyn's story, enhancing its journalistic quality.",
    
    "Comparing the rewritten text to the original text, the rewrite prompt transforms the passage into spoken word poetry, characterized by rhythmic language, vivid imagery, and emotional depth. It maintains the essence of Lorraine's experience of joy and hope while enhancing it with poetic elements like metaphor and rhythm.",
    "Comparing the rewritten text to the original text, the rewrite prompt transforms the passage into a sestina, characterized by the repetition of certain words and phrases in a structured manner. It retains the essential information about Uncle John's deathwatch while adhering to the form's rules and poetic structure.",
    "Comparing the rewritten text to the original text, the rewrite prompt transforms the passage into a fable-like narrative, imbuing it with a sense of wonder and whimsy. It retains the essence of Stephen's fascination with astronomy while adding elements of storytelling, metaphor, and a timeless quality reminiscent of traditional fables.",
    "Comparing the rewritten text to the original text, the rewrite prompt transforms the passage into a dialogue by introducing two characters, Christopher and the speaker (referred to as \"You\"). It maintains the request for clarification regarding the new software while adding conversational elements and interaction between the characters.",

    "Comparing the rewritten text to the original text, the rewrite prompt replaces the characters with those from Lord of the Rings, aligning with the directive to rewrite the passage with characters from a specific context. The essence of sharing information remains, albeit with a thematic shift towards a different fictional universe.",
    "Comparing the rewritten text to the original text, the rewrite prompt aligns with the Stranger Things series by infusing it with the show's tone and atmosphere. It transforms the mundane act of purchasing a peacoat into a dramatic and exhilarating scene reminiscent of the series' adventurous spirit and 80s nostalgia.",
    "Comparing the rewritten text to the original text, the rewrite prompt reframes the passage into the tone of The Hobbit book by incorporating imagery and language reminiscent of the fantasy genre. It transforms Lee's entrepreneurial journey into an epic quest, using metaphors of adventure and heroism to convey his determination and eventual success.",
    "Comparing the rewritten text to the original text, the rewrite prompt aligns with the tone of the Inception movie by infusing it with slang, colloquial language, and a sense of casualness. It transforms the mundane conversation into a more dynamic and expressive dialogue, reminiscent of the film's style and atmosphere.",

    "Comparing the rewritten text to the original text, the rewrite prompt completely changes the context from a thunderstorm gathering to a cooking recipe. It transforms the narrative into step-by-step instructions for preparing a dish, utilizing ingredients and cooking techniques instead of describing a social event.",
    "Comparing the rewritten text to the original text, the rewrite prompt transforms the passage into a speech by imbuing it with formal language and rhetorical elements. It retains the essence of Kenneth's quest for the perfect peacoat while elevating it into a persuasive and eloquent address to an audience.",
    "Comparing the rewritten text to the original text, the rewrite prompt adapts the passage into a political commentary by using metaphorical language and drawing parallels between Richard's fishing experience and the American political landscape. It retains the essence of Richard's success while framing it within the context of political ideologies and societal issues.",
    "Comparing the rewritten text to the original text, the rewrite prompt reframes the description of Martin's room into a financial analysis by focusing on its potential return on investment. It emphasizes the value of the items in Martin's room in terms of tenant attraction, reduced vacancy, and increased productivity, aligning with a financial perspective rather than mere description.",
]

total_df['reasoning'] = reasoning_ls
total_df.to_csv("./data/0408/data/categorized_seed.csv", index=False)


In [26]:
df_shuffle = df.iloc[subset].sample(frac=1).reset_index(drop=True)
df_shuffle.to_csv("./data/0401/nbroad-v2-gemma_suppl_rewrite.csv", index=False, columns=['rewrite_prompt'])

df_shuffle.iloc[0]['rewrite_prompt']

'Refine this for a financial analysis.'

# Build predict datasets

In [27]:
import random

random.seed(42)

# df_prompts = pd.read_csv("./data/0401/nbroad-v2-gemma_suppl_rewrite.csv")
df_text = pd.read_csv("./data/0401/150_suppl_original_text.csv")
df_text = df_text[df_text['original_text'].str.split(" ").apply(len) <= 100]
df_text = df_text[~df_text['original_text'].str.contains(r"\[\w+\]", regex=True)]

# prompts = df_prompts.iloc[:200]['rewrite_prompt'].tolist()

df_rows = []
for i, row in df_prompts.iterrows():
    texts = df_text.iloc[random.choices(range(len(df_text)), k=2)]['original_text'].to_list()
    for t in texts:
        df_rows.append({'rewrite_prompt': row['prompt'], 'original_text': t, 'type': row['type']})

df = pd.DataFrame(df_rows)
df = df.sample(frac=1).reset_index(drop=True)

df.to_csv("./data/0407/diverse_dataset.csv", index=False)

df.shape

NameError: name 'df_prompts' is not defined